In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import seaborn

Import and sotering of log

In [2]:
log_df = pd.read_csv('log.csv', sep=';', usecols=['EventName','ID'])

# Function to parse the DCR XML and extract rules
def parse_dcr_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
log_df = log_df.groupby('ID')['EventName'].apply(list)

print(log_df)
print("HEHE\n", log_df[[3]])

ID
14a-026_0    [Fill out application, Approved - to board, Ch...
14a-027_0    [Fill out application, Approved - to board, Ch...
14a-031_2    [Fill out application, Screening reject, Appli...
14a-035_0    [Fill out application, Approved - to board, Ch...
14a-037_0    [Fill out application, Approved - to board, Ch...
                                   ...                        
14b-627_0    [Fill out application, Approved - to board, Ch...
14b-627_1    [Fill out application, Approved - to board, Ch...
14b-628_0    [Fill out application, Approved - to board, Ch...
14b-629_1    [Fill out application, Approved - to board, Ch...
14b-631_0    [Fill out application, Approved - to board, Ch...
Name: EventName, Length: 594, dtype: object
HEHE
 ID
14a-035_0    [Fill out application, Approved - to board, Ch...
Name: EventName, dtype: object


Hardcoded conformance check

In [3]:
def check_conformance(log_df):
    
    
    rule_counts = {
        "Rule 1": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 2": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 3": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 4": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 5": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 6": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 7": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
        "Rule 8": {"Satisfied": 0, "Not Satisfied": 0, "Irrelevant": 0},
    }

    
    

# Iterate through each event in the log and check rules    
    for events in log_df:
            for i, event in enumerate(events):
                # Rule 1: 'Fill out application' should always be the first event
                if event == "Fill out application":
                    if i == 0:
                        rule_counts["Rule 1"]["Satisfied"] +=1
                    else:
                        rule_counts["Rule 1"]["Not Satisfied"] += 1
                     
            # Rule 2: 'Lawyer Review' and 'Architect Review' should never occur together
            if "Lawyer Review" in events and "Architect Review" in events:
                rule_counts["Rule 2"]["Not Satisfied"] += 1
            elif "Lawyer Review" in events:
                rule_counts["Rule 2"]["Satisfied"] += 1
            elif "Architect Review" in events:
                rule_counts["Rule 2"]["Satisfied"] += 1
            else:
                rule_counts["Rule 2"]["Irrelevant"] += 1

            # Rule 3: Reject should always eventually be followed by Applicant informed and Change phase to Abort. 
            if "Reject" in events:
                reject_index = events.index("Reject")
                if "Applicant informed" not in events[reject_index:] or "Change phase to Abort" not in events[reject_index:]:
                    rule_counts["Rule 3"]["Not Satisfied"] += 1
                else:
                    rule_counts["Rule 3"]["Satisfied"] += 1

            # Rule 4: First payment should only occur once, unless Undo payment is executed afterwards, in which case it may be repeated once.
            first_payment_count = events.count("First payment")
            if first_payment_count == 1 or (first_payment_count == 2 and "Undo payment" in events):
                rule_counts["Rule 4"]["Satisfied"] += 1
            elif first_payment_count == 0:
                rule_counts["Rule 4"]["Satisfied"] += 1  # Satisfies the rule by default
            elif first_payment_count not in [0, 1, 2]:
                rule_counts["Rule 4"]["Not Satisfied"] += 1
            else:
                rule_counts["Rule 4"]["Not Satisfied"] += 1

            # Check if "First payment" never occurred, it satisfies the rule by default
            #if first_payment_count == 0:
                #rule_counts["Rule 4"]["Satisfied"] += 1
        





    # Print or return the counts
    print("This is what events mean:",events)
    print("This is what event means:",event)
    return rule_counts

# conformance check
result = check_conformance(log_df)
print("Conformance Check Results:")
for rule, counts in result.items():
    print(f"{rule}: Satisfied {counts['Satisfied']}, Not Satisfied {counts['Not Satisfied']}, Irrelevant {counts['Irrelevant']}")

This is what events mean: ['Fill out application', 'Approved - to board', 'Change phase to Review', 'Inform application of board review', 'Architect Review', 'Review', 'Review', 'Register Decision', 'Change phase to Board meeting', 'Lawyer Review', 'Approve', 'Change phase to Preparation', 'Set to Pre-approved', 'Screen application', 'Execute pre-decision', 'Approve', 'Execute pre-decision', 'Change phase to Preparation', 'Set to Pre-approved', 'Inform applicant of approval', 'Round approved', 'Round ends', 'Round approved']
This is what event means: Round approved
Conformance Check Results:
Rule 1: Satisfied 594, Not Satisfied 15, Irrelevant 0
Rule 2: Satisfied 1, Not Satisfied 289, Irrelevant 304
Rule 3: Satisfied 225, Not Satisfied 0, Irrelevant 0
Rule 4: Satisfied 594, Not Satisfied 0, Irrelevant 0
Rule 5: Satisfied 0, Not Satisfied 0, Irrelevant 0
Rule 6: Satisfied 0, Not Satisfied 0, Irrelevant 0
Rule 7: Satisfied 0, Not Satisfied 0, Irrelevant 0
Rule 8: Satisfied 0, Not Satisfie

Parsing and defining xml file

In [4]:
def parse_xml_file(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    print("Root tag:", root.tag)
    for child in root.findall("labelMapping"):
        print("Child tag:", child.tag)

    mapping = dict()
    for activity in root.find("specification").find("resources").find("labelMappings").findall("labelMapping"):
        #print(activity.get("labelId"))
        #print(activity.get("eventId"))
        mapping[activity.get("labelId")] =activity.get("eventId") 
    print(mapping)
xml_file_path = "DCR-Task-2.xml"

parse_xml_file(xml_file_path)

Root tag: dcrgraph
{'Fill out application': 'Activity0', 'Lawyer review': 'Activity1', 'Architect Review': 'Activity2', 'Change phase to abandon': 'Activity3', 'Execute abandon': 'Activity4', 'Approve': 'Activity5', 'Reject': 'Activity6', 'Approve changed account': 'Activity7', 'Account number changed': 'Activity8', 'Change phase to payout': 'Activity9', 'First payment': 'Activity10', 'Undo Payment': 'Activity11', 'Change phase to aborted': 'Activity12', 'Applicant informed': 'Activity13', 'Change phase to end report': 'Activity14', 'Receive end report': 'Activity15', 'Change phase to completed': 'Activity16'}


Conformance check 

In [5]:

#print(log_df)
#parse_xml_file(xml_file_path)

pattern_counts = {
        "Pattern 1": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 2": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 3": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 4": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 5": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 6": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 7": {"Satisfied": 0, "Not Satisfied": 0,},
        "Pattern 8": {"Satisfied": 0, "Not Satisfied": 0,},
    }


def parse_respons(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()



    mapping2 = dict()
    for activity in root.find("specification").find("constraints").find("responses").findall("response"):
    #print(activity.get("labelId"))
    #print(activity.get("eventId"))
        mapping2[activity.get("sourceId")] = activity.get("targetId") 
    print(mapping2)

parse_respons(xml_file_path)


def pattern1_response(event, rules):
    if event in rules['responses']:
        return rule
    return None



{'Activity0': 'Activity2', 'Activity1': 'Activity6', 'Activity2': 'Activity6', 'Activity6': 'Activity12', 'Activity5': 'Activity8', 'Activity4': 'Activity3', 'Activity15': 'Activity16', 'Activity8': 'Activity7', 'Activity7': 'Activity9', 'Activity9': 'Activity10', 'Activity10': 'Activity14', 'Activity14': 'Activity15'}
